# **LINEARISATION UNIQUE DES TRAFICS SUR BORDEAUX METROPOLE**
> Traitements à mettre en oeuvre

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np
import re
from difflib import get_close_matches,SequenceMatcher
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from collections import Counter
pd.set_option('display.max_columns', 100)
#pd.set_option('display.max_rows', 200)

## 1. MISE EN FORME DES COMPTAGES PONCTUELS

In [2]:
#import des données
cpt_pct=pd.read_excel(r'Q:\DAIT\TI\Bordeaux_metropole\2019\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\base_comptages_routiers_v4.xlsx')
cpt_pct.columns=[a.lower() for a in cpt_pct.columns]
cpt_pct = gp.GeoDataFrame(cpt_pct, geometry=gp.points_from_xy(cpt_pct.longitude, cpt_pct.latitude))
cpt_pct.crs = {'init' :'epsg:4326'}
cpt_pct_l93=cpt_pct.to_crs({'init': 'epsg:2154'})
cpt_pct_l93['x_l93']=cpt_pct_l93.geometry.apply(lambda x : x.x)
cpt_pct_l93['y_l93']=cpt_pct_l93.geometry.apply(lambda x : x.y)

In [3]:
#mise en forme
cpt_pct_l93['nom_voie']=cpt_pct_l93.nom_voie.apply(lambda x : re.sub(('é|è|ê'),'e',x.lower()))
cpt_pct_l93['sens_unique']=cpt_pct_l93.sens_circulation.apply(lambda x : True if SequenceMatcher(None,' '.join(x.split(' ')[:2]).lower(),'sens unique').ratio()>0.8 else False)
cpt_pct_l93['type_voie']=cpt_pct_l93.nom_voie.apply(lambda x : x.split(' ')[0])
cpt_pct_l93['suffix_nom_voie']=cpt_pct_l93.nom_voie.apply(lambda x : ' '.join(x.split(' ')[1:]).lower())

In [4]:
#cluster à 200m
data_test_clust=[[x, y] for x, y in zip(cpt_pct_l93.x_l93.tolist(), cpt_pct_l93.y_l93.tolist())]
db = DBSCAN(eps=200, min_samples=2).fit(data_test_clust)
labels = db.labels_
cpt_pct_l93['n_cluster']=labels

In [5]:
#voies avec nom_proches
cross_join_ncluster=cpt_pct_l93[['ident','nom_voie','type_voie','suffix_nom_voie', 'sens_circulation','annee','n_cluster']].merge(
   cpt_pct_l93[['ident','nom_voie','type_voie','suffix_nom_voie', 'sens_circulation','annee','n_cluster']], on='n_cluster') #avoir toutes les relations internoms possibles
cross_join_ncluster['comp_nom_voie']=cross_join_ncluster.apply(lambda x : SequenceMatcher(None,x['suffix_nom_voie_x'], x['suffix_nom_voie_y']).ratio(), axis=1)#affecter une note a cahque relation
voie_nom_proches=cross_join_ncluster.loc[(cross_join_ncluster['comp_nom_voie']>0.85) & (cross_join_ncluster['type_voie_x']==cross_join_ncluster['type_voie_y'])
                        ].sort_values(['n_cluster','ident_x'])#ne conserver qque les relations bien notees
voie_nom_proches['id_grp_nom_voie']=voie_nom_proches.ident_x.rank(method='dense')#ajouter un id 
corresp_voies=voie_nom_proches.drop_duplicates('ident_y')
grp_nom_voie=cpt_pct_l93.merge(corresp_voies[['ident_y','id_grp_nom_voie']].rename(columns={'ident_y':'ident'}), on='ident')#jointure sur id de depart

In [35]:
grp_period[[a for a in grp_period.columns if a!='date_max_cptg']].to_file(r'Q:\DAIT\TI\Bordeaux_metropole\2019\C19SA0101\Doc_travail\Donnees_produites\Donnees\cluster_comptg_pctl_grp_period.shp')

In [39]:
#grouper par periode
grp_period=grp_nom_voie.copy()
grp_period['date_max_cptg']=grp_period.observation.apply(lambda x : pd.to_datetime(x.split(' au ')[1],dayfirst=True))
grp_period['id_period']=grp_period.date_max_cptg.rank(method='dense')
grp_period['indicefinal']=grp_period.n_cluster**2+grp_period.id_grp_nom_voie**3+grp_period.id_period**4
grp_period=grp_period.assign(id_grp_period=grp_period.groupby(['n_cluster','id_grp_nom_voie','date_max_cptg'])['indicefinal'].rank(method='dense'))

In [40]:
grp_period

,ident,type_compteur,latitude,longitude,commune_code_insee,nom_voie,sens_circulation,tmjo_tv,tmjo_vl,tmjo_pl,hpm_tv,hps_tv,v85_vl,v85_pl,annee,observation,geometry,x_l93,y_l93,sens_unique,type_voie,suffix_nom_voie,n_cluster,id_grp_nom_voie,date_max_cptg,id_period,indicefinal,id_grp_period
0,1,comptage_ponctuel,44.843816,-0.582561,63,rue thiac,Sens unique vers la rue Rodrigues Pereire,276,276,0,31,29,NC,NC,2015,comptage du 20/11/2015 au 26/11/2015,POINT (417006.507 6422492.898),417006.506742,6.422493e+06,True,rue,thiac,0,1.0,2015-11-26,59.0,1.211736e+07,1.0
1,2,comptage_ponctuel,44.843671,-0.584986,63,rue rodrigues pereire,Sens unique vers la rue Saint Etienne,2365,2365,0,183,242,NC,NC,2015,comptage du 20/11/2015 au 26/11/2015,POINT (416814.352 6422485.503),416814.351901,6.422486e+06,True,rue,rodrigues pereire,0,2.0,2015-11-26,59.0,1.211737e+07,1.0
2,3,comptage_ponctuel,44.843663,-0.585770,63,rue saint etienne,Sens unique vers la rue Concorde,786,786,0,59,94,NC,NC,2015,comptage du 20/11/2015 au 26/11/2015,POINT (416752.424 6422487.428),416752.424312,6.422487e+06,True,rue,saint etienne,0,3.0,2015-11-26,59.0,1.211739e+07,1.0
3,4,comptage_ponctuel,44.842492,-0.583581,63,rue casteja,Sens unique vers la rue Abbé de l'Epée,4869,4869,0,427,393,NC,NC,2015,comptage du 20/11/2015 au 26/11/2015,POINT (416919.319 6422349.649),416919.318622,6.422350e+06,True,rue,casteja,0,4.0,2015-11-26,59.0,1.211742e+07,1.0
4,5,comptage_ponctuel,44.843077,-0.583281,63,rue abbe de l'epee,Vers la rue Castéja,1967,1967,0,258,177,NC,NC,2015,comptage du 20/11/2015 au 26/11/2015,POINT (416945.948 6422413.482),416945.948064,6.422413e+06,False,rue,abbe de l'epee,0,5.0,2015-11-26,59.0,1.211749e+07,1.0
5,6,comptage_ponctuel,44.842910,-0.583109,63,rue abbe de l'epee,Vers la rue Thiac,913,913,0,88,86,NC,NC,2015,comptage du 20/11/2015 au 26/11/2015,POINT (416958.684 6422394.336),416958.684127,6.422394e+06,False,rue,abbe de l'epee,0,5.0,2015-11-26,59.0,1.211749e+07,1.0
6,7,comptage_ponctuel,44.834297,-0.577165,63,rue du commandant arnoult,Sens unique vers la place Pey Berland,855,838,17,55,56,36,36,2015,comptage du 22/01/2015 au 28/01/2015,POINT (417384.575 6421417.380),417384.574943,6.421417e+06,True,rue,du commandant arnoult,1,7.0,2015-01-28,3.0,4.250000e+02,1.0
7,8,comptage_ponctuel,44.846549,-0.613325,63,rue bahr,Sens unique vers la rue Charles Chaumet,4587,4500,87,442,406,38.4,40,2015,comptage du 02/03/2015 au 08/03/2015,POINT (414591.973 6422906.863),414591.973298,6.422907e+06,True,rue,bahr,-1,8.0,2015-03-08,12.0,2.124900e+04,1.0
8,9,comptage_ponctuel,44.822786,-0.602789,63,rue de bethmann,Vers la rue Beira,7483,7234,249,727,751,57.1,55.5,2015,comptage du 02/03/2015 au 08/03/2015,POINT (415303.225 6420232.216),415303.225103,6.420232e+06,False,rue,de bethmann,2,9.0,2015-03-08,12.0,2.146900e+04,1.0
9,10,comptage_ponctuel,44.822701,-0.603420,63,rue de bethmann,Vers la rue du Tauzin,6811,6604,207,549,564,52.4,49.2,2015,comptage du 02/03/2015 au 08/03/2015,POINT (415252.967 6420225.060),415252.966642,6.420225e+06,False,rue,de bethmann,2,9.0,2015-03-08,12.0,2.146900e+04,1.0


In [31]:
#groupement final
grp_final=grp_period.copy()
grp_final.groupby()

ident                          int64
type_compteur                 object
latitude                     float64
longitude                    float64
commune_code_insee             int64
nom_voie                      object
sens_circulation              object
tmjo_tv                        int64
tmjo_vl                       object
tmjo_pl                       object
hpm_tv                        object
hps_tv                        object
v85_vl                        object
v85_pl                        object
annee                          int64
observation                   object
geometry                    geometry
x_l93                        float64
y_l93                        float64
sens_unique                     bool
type_voie                     object
suffix_nom_voie               object
n_cluster                      int64
id_grp_nom_voie              float64
date_max_cptg         datetime64[ns]
id_grp_period                float64
dtype: object

In [24]:
grp_period.observation.apply(lambda x : pd.to_datetime(x.split(' au ')[1],dayfirst=True))

0      2015-11-26
1      2015-11-26
2      2015-11-26
3      2015-11-26
4      2015-11-26
5      2015-11-26
6      2015-01-28
7      2015-03-08
8      2015-03-08
9      2015-03-08
10     2015-06-02
11     2015-06-02
12     2015-09-16
13     2015-09-16
14     2015-09-16
15     2015-09-16
16     2015-09-16
17     2015-09-16
18     2015-09-16
19     2015-09-16
20     2015-09-16
21     2015-09-16
22     2015-09-16
23     2015-09-16
24     2015-09-16
25     2015-07-05
26     2015-07-05
27     2015-06-03
28     2015-06-03
29     2015-06-05
          ...    
1477   2018-03-20
1478   2018-03-20
1479   2018-03-20
1480   2018-03-20
1481   2018-03-20
1482   2018-03-20
1483   2018-03-20
1484   2018-03-20
1485   2018-03-20
1486   2018-03-20
1487   2018-03-20
1488   2018-03-20
1489   2018-03-20
1490   2018-03-22
1491   2018-03-22
1492   2018-03-23
1493   2018-03-22
1494   2018-03-22
1495   2018-03-22
1496   2018-03-22
1497   2018-03-22
1498   2018-03-22
1499   2018-04-01
1500   2018-04-01
1501   201

## 2. MISE EN FORME DU MMM

En premier lieu il faut mettre à jour les attributs en renomant de façon explicite puis en faisant la somme des 2 sens de circulation pour les voies doucble sens

In [4]:
#importer les données et convertir les noms de champs
fichier_src=gp.read_file(r'Q:\DAIT\TI\Bordeaux_metropole\2019\C19SA0101\Doc_travail\Donnees_produites\QGIS\tables\MMM_BdxM.shp')
#définir les colonnes définitives (PLUS TARD ON FERA UN FILTRE EN AMONT)
liste_col_def=['NO', 'FROMNODENO', 'TONODENO', 'TYPENO', 'TSYSSET', 'LENGTH', 'NUMLANES', 'CAPPRT', 'V0_TV', 'VOLVEHPR~1', 
'Q_HC_PL', 'Q_HC_TV', 'Q_HC_VL', 'Q_HPM_PL', 'Q_HPM_TV', 'Q_HPM_VL', 'Q_HPS_PL', 'Q_HPS_TV', 'Q_HPS_VL', 'Q_Jour_PL', 'Q_Jour_TV',
'Q_Jour_VL','coef_HC_PL','coef_HC_TV','coef_HC_VL','Ocup_VL_PL','V0_PL','V0_VL','VCharg_PL','VCharg_VL','VOLPCUP~22',
'SHAREHGV', 'VOLVEH_~23', 'VOLVEH_~24', 'VOLCAPR~25', 'VEHHOUR~26', 'SPEEDLIMIT',
'R_NO', 'R_FROMNODENO', 'R_TONODENO', 'R_TYPENO', 'R_TSYSSET','R_LENGTH', 'R_NUMLANES', 'R_CAPPRT', 'r_V0_TV', 'R_VOLVEHPR~1', 
'R_Q_HC_PL', 'R_Q_HC_TV', 'R_Q_HC_VL', 'R_Q_HPM_PL', 'R_Q_HPM_TV', 'R_Q_HPM_VL', 
'R_Q_HPS_PL', 'R_Q_HPS_TV', 'R_Q_HPS_VL', 'R_Q_Jour_PL', 'R_Q_Jour_TV', 'R_Q_Jour_VL', 'R_coef_HC_PL', 'R_coef_HC_TV', 'R_coef_HC_VL', 'R_Ocup_VL_PL', 'R_V0_PL', 
'R_V0_VL', 'R_VCharg_PL', 'R_VCharg_VL', 'R_VOLPCUP~22', 'R_SHAREHGV', 'R_VOLVEH_~23', 'R_VOLVEH_~24', 'R_VOLCAPR~25', 'R_VEHHOUR~26', 'R_SPEEDLIMIT']
#creer un dico de renomage
dico_renomage={a: b for a, b in zip(fichier_src.columns,liste_col_def)}
#renommer
fichier_src.rename(columns=dico_renomage, inplace=True)

In [21]:
#ne conserver dans un premier temps que les attributs relatifs au trafic sur la journée
fichier_src_simpl=fichier_src[['NO', 'FROMNODENO', 'TONODENO', 'TYPENO', 'TSYSSET', 'LENGTH', 'NUMLANES', 'CAPPRT', 'V0_TV', 
             'VOLVEHPR~1']+[a for a in fichier_src.columns if 'Jour' in a ]].copy()

In [30]:
#caculer le tmja_TV
fichier_src_simpl['tmja_tv']=fichier_src_simpl.Q_Jour_TV+fichier_src_simpl.R_Q_Jour_TV

In [28]:
fichier_src_simpl.head(5)

,NO,FROMNODENO,TONODENO,TYPENO,TSYSSET,LENGTH,NUMLANES,CAPPRT,V0_TV,VOLVEHPR~1,Q_Jour_PL,Q_Jour_TV,Q_Jour_VL,R_Q_Jour_PL,R_Q_Jour_TV,R_Q_Jour_VL,tmja_tv
0,2000062,2000054,5868961,82,"A,B,BHNS,P,R,V",0.114km,1,600,40km/h,314,221.68,4422.38,4200.70,294.99,5945.08,5650.08,10367.46
1,2000063,2000054,6389027,82,"A,B,BHNS,P,R,V",0.099km,1,600,40km/h,467,294.99,5945.08,5650.08,221.68,4422.38,4200.70,10367.46
2,2000064,3527,2000055,77,"A,B,BHNS,P,R,V",0.160km,1,750,40km/h,269,582.93,9835.65,9190.25,424.10,6218.11,5755.06,16053.76
3,2000065,2000055,4852848,77,"A,B,BHNS,P,R,V",0.037km,1,750,40km/h,269,582.93,9835.65,9190.25,424.10,6218.11,5755.06,16053.76
4,2000068,5743212,20000010,82,"A,B,BHNS,P,R,V",0.074km,1,600,40km/h,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
